# Chapter 06 파이썬 고급 주제

## 6.1 멀티 프로세스와 멀티 스레드
* 멀티 프로세스
* 멀티 스레드

### 6.1.1 subprocess 모듈
subprocess 모듈은 '부모-자식' 프로세스 쌍을 생성하는 데 사용

In [3]:
import subprocess
subprocess.run(["echo", "이것은 subprocess입니다."])

subprocess.run(["sleep", "10"]) # 10초 동안 sleep

CompletedProcess(args=['sleep', '10'], returncode=0)

### 6.1.2 threading 모듈
스레드가 여러 개로 분리되면, 스레드 간 데이터 공유의 복잡성이 증가. 내부적으로 락을 관리하기 위해 queue 모듈 사용. 밑에서 queue 모듈의 공식 문서 예제 수정.

In [7]:
import queue
import threading

q = queue.Queue()

def worker(num):
    while True:
        item = q.get()
        if item is None:
            break
        # 작업을 처리한다.
        print(f"스레드 {num+1} : 처리 완료 {item}")
        q.task_done()

if __name__ == "__main__":
    num_worker_threads = 5
    threads = []
    for i in range(num_worker_threads):
        t = threading.Thread(target=worker, args=(i,))
        t.start()
        threads.append(t)

    for item in range(20):
        q.put(item)

    # 모든 작업이 끝날 때까지 대기한다(block).
    q.join()

    # 워커 스레드를 종료한다(stop).
    for i in range(num_worker_threads):
        q.put(None)
    for t in threads:
        t.join()

스레드 1 : 처리 완료 0
스레드 1 : 처리 완료 1
스레드 1 : 처리 완료 2스레드 3 : 처리 완료 3
스레드 2 : 처리 완료 4
스레드 2 : 처리 완료 5
스레드 2 : 처리 완료 6
스레드 2 : 처리 완료 7
스레드 2 : 처리 완료 8
스레드 2 : 처리 완료 9
스레드 2 : 처리 완료 10
스레드 2 : 처리 완료 11
스레드 2 : 처리 완료 12
스레드 2 : 처리 완료 13
스레드 2 : 처리 완료 14
스레드 2 : 처리 완료 15
스레드 2 : 처리 완료 16
스레드 2 : 처리 완료 17
스레드 1 : 처리 완료 18
스레드 1 : 처리 완료 19



### 6.1.3 뮤텍스와 세마포어
mutex는 락과 같음. 1부터 시작하는 정수

In [11]:
from threading import Thread, Lock
import threading

def worker(mutex, data, thread_safe):
    if thread_safe:
        mutex.acquire()
    try:
        print(f"스레드 {threading.get_ident()}: {data}\n")
    finally:
        if thread_safe:
            mutex.release()
    
if __name__ == "__main__":
    threads = []
    thread_safe = True   # 뮤텍스 사용시 True
    mutex = Lock()
    for i in range(20):
        t = Thread(target=worker, args=(mutex, i, thread_safe))
        t.start()
        threads.append(t)
    for t in threads:
        t.join()

스레드 140466045925120: 0

스레드 140466037532416: 1

스레드 140466062710528: 2

스레드 140466140653312: 3

스레드 140466037532416: 4

스레드 140466062710528: 5

스레드 140466045925120: 6

스레드 140466054317824: 7

스레드 140466037532416: 8

스레드 140466140653312: 9

스레드 140466029139712: 10

스레드 140466062710528: 11

스레드 140466045925120: 12

스레드 140466029139712: 13

스레드 140466062710528: 14

스레드 140466045925120: 15

스레드 140466045925120: 16

스레드 140466029139712: 17

스레드 140466062710528: 18

스레드 140466062710528: 19



세마포어는 뮤텍스보다 더 일반적으로 사용되는 개념. 1보다 큰 수로 시작 가능. 세마포어값은 즉 한 번에 자원에 접근할 수 있는 스레드의 수다. 대기 및 신호 작업 지원

In [12]:
import threading
import time

class ThreadPool(object):
    def __init__(self):
        self.active = []
        self.lock = threading.Lock()

    def acquire(self, name):
        with self.lock:
            self.active.append(name)
            print(f"획득: {name} | 스레드 풀: {self.active}")
    
    def release(self, name):
        with self.lock:
            self.active.remove(name)
            print(f"반환: {name} | 스레드 풀: {self.active}")
    
def worker(semaphore, pool):
    with semaphore:
        name = threading.currentThread().getName()
        pool.acquire(name)
        time.sleep(1)
        pool.release(name)

if __name__ == "__main__":
    threads = []
    pool = ThreadPool()
    semaphore = threading.Semaphore(3)
    for i in range(10):
        t = threading.Thread(
            target=worker, name="스레드 " + str(i), args=(semaphore, pool))
        t.start()
        threads.append(t)
    for t in threads:
        t.join()

획득: 스레드 0 | 스레드 풀: ['스레드 0']
획득: 스레드 1 | 스레드 풀: ['스레드 0', '스레드 1']
획득: 스레드 2 | 스레드 풀: ['스레드 0', '스레드 1', '스레드 2']
반환: 스레드 0 | 스레드 풀: ['스레드 1', '스레드 2']
반환: 스레드 1 | 스레드 풀: ['스레드 2']
반환: 스레드 2 | 스레드 풀: []
획득: 스레드 3 | 스레드 풀: ['스레드 3']
획득: 스레드 5 | 스레드 풀: ['스레드 3', '스레드 5']
획득: 스레드 4 | 스레드 풀: ['스레드 3', '스레드 5', '스레드 4']
반환: 스레드 3 | 스레드 풀: ['스레드 5', '스레드 4']
반환: 스레드 5 | 스레드 풀: ['스레드 4']
획득: 스레드 7 | 스레드 풀: ['스레드 4', '스레드 7']
획득: 스레드 6 | 스레드 풀: ['스레드 4', '스레드 7', '스레드 6']
반환: 스레드 4 | 스레드 풀: ['스레드 7', '스레드 6']
획득: 스레드 8 | 스레드 풀: ['스레드 7', '스레드 6', '스레드 8']
반환: 스레드 7 | 스레드 풀: ['스레드 6', '스레드 8']
반환: 스레드 6 | 스레드 풀: ['스레드 8']
획득: 스레드 9 | 스레드 풀: ['스레드 8', '스레드 9']
반환: 스레드 8 | 스레드 풀: ['스레드 9']
반환: 스레드 9 | 스레드 풀: []


### 6.1.4 데드락과 스핀락
데드락(교착 상태)은 두 개 이상의 프로세스나 스레드가 서로 상대방의 작업이 끝나기만을 기다리고 있기 때문에 결과적으로 아무것도 완료되지 못하는 상태. 프로그램에 락을 할당하고, 락을 순서대로 획득한다면, 교착 상태를 막을 수 있다.<br/>
다음 네 가지 조건을 모두 충족하면 데드락 발생
* 상호 배제
* 점유와 대기
* 비선점
* 순환 대기 <br/>

스핀락은 고성능 컴퓨팅 상황에 유용한 바쁜대기의 한 형태. 임계 구역에 진입이 불가능할 때, 진입이 가능할 때까지 반복문을 돌면서 재시도하는 방식으로 구현된 락.

### 6.1.5 스레딩에 대한 구글 파이썬 스타일 가이드

저수준의 락 대신, threading.Condition을 사용할 수 있도록 조건 변수를 적절하게 사용하는 방법을 숙지. 아래는 생산자-소비자 모델의 간단한 예.

In [13]:
import threading

def consumer(cond):
    name = threading.currentThread().getName()
    print(f"{name} 시작")
    with cond:
        print(f"{name} 대기")
        cond.wait()
        print(f"{name} 자원 소비")

def producer(cond):
    name = threading.currentThread().getName()
    print(f"{name} 시작")
    with cond:
        print(f"{name} 자원 생산 후 모든 소비자에게 알림")
        cond.notifyAll()

if __name__ == "__main__":
    condition = threading.Condition()
    consumer1 = threading.Thread(
        name="소비자1", target=consumer, args=(condition,))
    consumer2 = threading.Thread(
        name="소비자2", target=consumer, args=(condition,))
    producer = threading.Thread(name="생산자", target=producer,
                                args=(condition,))
    
    consumer1.start()
    consumer2.start()
    producer.start()

소비자1 시작
소비자2 시작
소비자2 대기
소비자1 대기
생산자 시작
생산자 자원 생산 후 모든 소비자에게 알림
소비자1 자원 소비
소비자2 자원 소비
